# Set parameters for script

In [ ]:
$resourcegroup = 'aks-rg-demo'
$aksclustername = 'aks-cluster-demo'
$SubscriptionId = "xxxxx-Add Your Subscription Here-xxxxx"
$FolderLocation = "...\2 - Azure Kubernetes Service"
$cred = Get-Credential sa
Write-Output $resourcegroup $aksclustername $SubscriptionId $FolderLocation
Set-Location $FolderLocation
Get-ChildItem $FolderLocation

# Connect to Azure and set subscription

In [ ]:
Connect-AzAccount
az account show
az account set --subscription $SubscriptionId

## Start AKS Cluster

In [ ]:
az aks start --name $aksclustername --resource-group $resourcegroup

## Set location of Demo

In [ ]:
Set-Location $FolderLocation
Get-ChildItem

## Create Simple Upgrade deployment

In [ ]:
kubectl create namespace simpleupgrade

kubectl apply -f aks-sqlserver-nopvc.yaml -n simpleupgrade
#kubectl expose deployment aks-sqlserver-nopvc --name=sqlserverlb --type=LoadBalancer --port=1433 --target-port=1433 -n simpleupgrade

In [ ]:
kubectl get pods -n simpleupgrade
kubectl get service -n simpleupgrade
#$cred = Get-Credential sa

$podnamesimple = kubectl get pods -o jsonpath='{.items[0].metadata.name}' -n simpleupgrade
kubectl logs $podnamesimple -n simpleupgrade
kubectl describe pod $podnamesimple -n simpleupgrade

# Demo Start

## List K8s Contexts

In [ ]:
kubectl config get-contexts

## Switch Context to aks-cluster-demo

In [ ]:
kubectl config use-context $aksclustername #aks-cluster-demo

## Check Cluster Information

In [ ]:
kubectl cluster-info dump

## Check Namespaces

In [ ]:
kubectl get namespace

## Create "demo" Namespace

In [ ]:
kubectl create namespace demo

## View aks Nodes

In [ ]:
kubectl get nodes --namespace all

# Persisting Storage

## View Storage Class yaml

In [ ]:
Get-Content storageclass.yaml

## Create Storage Class

In [ ]:
kubectl apply -f storageclass.yaml

## Check Storage Class

In [ ]:
kubectl get storageclass
kubectl get sc

## View Persistent Volume Claim

In [ ]:
Get-Content persistentvolumeclaim.yaml

## Create Persistent Volume Claim

In [ ]:
kubectl apply -f persistentvolumeclaim.yaml -n demo

## Check Persistent Volume Claim

In [ ]:
kubectl get persistentvolumeclaim -n demo
kubectl get pvc -o wide -n demo

## View Persistent Volume Claim (Dynamically Generated)

In [ ]:
kubectl get persistentvolume -n demo
kubectl get pv -o wide -n demo

# Secrets

## Create Secret

In [ ]:
kubectl create secret generic secret-aks-mssql --from-literal=SA_PASSWORD="P@ssword1" -n demo

## View Secret

In [ ]:
kubectl get secrets -n demo

## Describe Secret

In [ ]:
kubectl describe secret secret-aks-mssql -n demo

# Create SQL Server On AKS

## Deploy SQL Server

In [ ]:
kubectl apply -f sqlserver_persistentvolume.yaml -n demo

In [ ]:
Get-Content sqlserver_persistentvolume.yaml

## Check Deployment

In [ ]:
# Can take up to 70s
kubectl get deployments -n demo

## View Pod Logs

In [ ]:
kubectl get pods -o wide -n demo
$podname = kubectl get pods -o jsonpath='{.items[0].metadata.name}' -n demo

In [ ]:
kubectl logs $podname -n demo

In [ ]:
kubectl describe pod $podname -n demo

## <span style="color: #FFFFFF;">Expose&nbsp;deployment to&nbsp;connect&nbsp;externally</span>

In [ ]:
#kubectl expose deployment aks-sqlserver --name=sqlservernp --type=NodePort --port=1433 --target-port=1433
kubectl expose deployment aks-sqlserver --name=sqlserverlb --type=LoadBalancer --port=1433 --target-port=1433 -n demo

In [ ]:
kubectl get service -n demo

# Creating / Persisting Databases

## Create Database and Login

In [ ]:
#$cred = Get-Credential sa
$podIP = kubectl get service -o jsonpath='{.items[0].status.loadBalancer.ingress[0].ip}' -n demo

write-host $podIP
write-host $cred

In [ ]:
Invoke-DbaQuery -SqlInstance $podIP -SqlCredential $cred -Query "select @@version"

In [ ]:
Invoke-DbaQuery -SqlInstance $podIP -SqlCredential $cred -Query "CREATE DATABASE Demo1;"

Invoke-DbaQuery -SqlInstance $podIP -SqlCredential $cred -Query "CREATE LOGIN [test1] WITH PASSWORD=N'P@ssword1', DEFAULT_DATABASE=[master], CHECK_EXPIRATION=OFF, CHECK_POLICY=OFF"

Invoke-DbaQuery -SqlInstance $podIP -SqlCredential $cred -Query "EXEC sys.sp_configure N'show advanced options', N'1'  RECONFIGURE WITH OVERRIDE
EXEC sys.sp_configure N'max server memory (MB)', N'3000'
RECONFIGURE WITH OVERRIDE
GO"

In [ ]:
Invoke-DbaQuery -SqlInstance $podIP -SqlCredential $cred -Query "select name AS DBName from sys.databases;"

In [ ]:
Invoke-DbaQuery -SqlInstance $podIP -SqlCredential $cred -Query "SELECT name AS LoginName FROM syslogins WHERE [name] like 'test%';"

## Have a bit play around to see if we can break things!

## Lets try Deleting a Pod

In [ ]:
$podname = kubectl get pods -o jsonpath='{.items[0].metadata.name}' -n demo
kubectl delete pod $podname -n demo

In [ ]:
kubectl get pods -o wide -n demo

In [ ]:
$podname = kubectl get pods -o jsonpath='{.items[0].metadata.name}' -n demo
kubectl logs $podname -n demo

In [ ]:
Invoke-DbaQuery -SqlInstance $podIP -SqlCredential $cred -Query "select name from sys.databases;"

## Lets go crazy and Delete the deployment!

In [ ]:
$FolderLocation = "C:\Users\ChrisTaylor\OneDrive\Git\Presentations\Docker\Kubernetify Your Containers\Data Relay 2021\Demo\2 - Azure Kubernetes Service"
Set-Location -Path $FolderLocation
kubectl delete deployment aks-sqlserver -n demo

In [ ]:
kubectl get deployments -n all
kubectl get pods -n demo

In [ ]:
kubectl apply -f sqlserver_persistentvolume.yaml -n demo

In [ ]:
# Can Take ~120s to fully come up due to detatch / re-attach persistentvolumeclaim's
kubectl get deployments -n demo

### Check DB still exits

In [ ]:
kubectl get pods -n demo

In [ ]:
$podname = kubectl get pods -o jsonpath='{.items[0].metadata.name}' -n demo
kubectl describe pod $podname -n demo

In [ ]:
$podIP = kubectl get service -o jsonpath='{.items[0].status.loadBalancer.ingress[0].ip}' -n demo
Invoke-DbaQuery -SqlInstance $podIP -SqlCredential $cred -Query "select name from sys.databases;"

## Try scaling to break things :)

In [ ]:
kubectl scale deployment/aks-sqlserver --replicas=3 -n demo

In [ ]:
kubectl get pods -o wide -n demo

The original Pod will still be ok

In [ ]:
$podname1 = kubectl get pods -o jsonpath='{.items[0].metadata.name}' -n demo
kubectl logs $podname1 -n demo

In [ ]:
kubectl describe pod $podname1 -n demo

2nd and 3rd Pods will fail to create

In [ ]:
$podname2 = kubectl get pods -o jsonpath='{.items[2].metadata.name}' -n demo
kubectl logs $podname2 -n demo
<#
 example error:
    Error 5(Access is denied.) occurred while opening file '/var/opt/mssql/data/master.mdf' to obtain configuration information at startup. 
    An invalid startup option might have caused the error. Verify your startup options, and correct or remove them if necessary.
#>

In [ ]:
kubectl scale deployment/aks-sqlserver --replicas=1 -n demo

In [ ]:
kubectl get pods -o wide -n demo

# Patching / Upgrading SQL Server

## Check SQL Server CU and Rollout History

We should be on SQL Server 2017 CU21 and looking to upgrade to CU23

In [ ]:
kubectl rollout history deployment/aks-sqlserver-nopvc -n simpleupgrade

In [ ]:
#kubectl get service -o jsonpath='{.items[0].status.loadBalancer.ingress[0].ip}' -n simpleupgrade

In [ ]:
#$podIP = kubectl get service -o jsonpath='{.items[0].status.loadBalancer.ingress[0].ip}' -n simpleupgrade
#write-host $podIP

In [ ]:
Invoke-DbaQuery -SqlInstance $podIP -SqlCredential $cred -Query "select @@version;"

## Lets upgrade to CU23

NOTE: There is more than one way to do this:

1. Re-Deploy
2. Switch the image used

In [ ]:
kubectl apply -f .\sqlserver_persistentvolume_2017_Upgrade_Simple.yaml -n simpleupgrade
#kubectl set image deployment/aks-sqlserver aks-sqlserver=mcr.microsoft.com/mssql/server:2017-CU23-ubuntu-16.04 -n demo
kubectl get pods -n simpleupgrade

In [ ]:
Get-Content .\sqlserver_persistentvolume_2017_Upgrade_Simple.yaml

In [ ]:
#kubectl delete pods --all -n demo
kubectl get pods -n simpleupgrade

## This will take anywhere from 1 - 5mins if using PVC

In [ ]:
$podname = kubectl get pods -o jsonpath='{.items[1].metadata.name}' -n simpleupgrade
kubectl describe pod $podname -n simpleupgrade
kubectl rollout history deployment/aks-sqlserver-nopvc -n simpleupgrade 

In [ ]:
kubectl logs $podname -n simpleupgrade

### Check that we are now on CU23 (can take a few minutes)

In [ ]:
#$cred = Get-Credential sa
$podIP = kubectl get service -o jsonpath='{.items[0].status.loadBalancer.ingress[0].ip}' -n simpleupgrade
Invoke-DbaQuery -SqlInstance $podIP -SqlCredential $cred -Query "select @@version;"

### Switch back to CU21 by using previous image

In [ ]:
#kubectl apply -f .\sqlserver_persistentvolume_2017.yaml
kubectl set image deployment/aks-sqlserver-nopvc aks-sqlserver-nopvc=ctaksacrdemo.azurecr.io/mssqldemo:2017-cu21 -n simpleupgrade
kubectl get pods -n simpleupgrade
kubectl rollout history deployment/aks-sqlserver-nopvc -n simpleupgrade

In [ ]:
$podname = kubectl get pods -o jsonpath='{.items[0].metadata.name}' -n simpleupgrade

In [ ]:
kubectl describe pod $podname -n simpleupgrade

In [ ]:
kubectl logs $podname -n simpleupgrade

### Final check of the pod to make sure we're back on CU21 (can also see in Azure Portal Replica Set)

In [ ]:
$podIP = kubectl get service -o jsonpath='{.items[0].status.loadBalancer.ingress[0].ip}' -n simpleupgrade
Invoke-DbaQuery -SqlInstance $podIP -SqlCredential $cred -Query "select @@version;"

# Remove everything from the Demo

In [ ]:
kubectl delete deployment aks-sqlserver -n demo
kubectl delete service aks-sqlserver-service -n demo
kubectl delete service sqlserverlb -n demo
kubectl delete secret secret-aks-mssql -n demo
kubectl delete pvc pvc-aks-mssql-systemdbs -n demo
kubectl delete pvc pvc-aks-mssql-data -n demo
kubectl delete pvc pvc-aks-mssql-logs -n demo
kubectl delete sc sc-aks-demo 

kubectl delete namespace demo
kubectl delete namespace simpleupgrade

In [ ]:
kubectl get deployment -n demo
kubectl get deployment -n simpleupgrade

Shutdown AKS Cluster (to save money!)

In [ ]:
<#
az account clear
az login
az account show
az account set --subscription $SubscriptionId
az aks get-credentials --name $aksclustername --resource-group $resourcegroup
#>
az aks stop --name $aksclustername --resource-group $resourcegroup

# Kubernetes Dashboard

In [ ]:
kubectl create clusterrolebinding kubernetes-dashboard --clusterrole=cluster-admin --serviceaccount=kube-system:kubernetes-dashboard

In [ ]:
az aks browse --resource-group $resourcegroup --name $aksclustername